In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
import joblib
import rembg


ModuleNotFoundError: No module named 'rembg'

## 광수 데이터 전처리 - 5

In [ ]:
data = np.load('kwangsoo.npz')
array1 = data['data1']
array1.shape

### 4차원으로 배열변환

In [5]:
array1=array1.reshape(438,160, 340, 4)
array1.shape

ValueError: cannot reshape array of size 1031640000 into shape (438,160,340,4)

### 그레이스케일 후 이미지를 1차원으로 벡터로 변환

In [93]:
a=[]
for i in array1:
    gray = cv2.cvtColor(i, cv2.COLOR_RGBA2GRAY)
    gray=gray.reshape(-1)
    a.append(gray)

In [94]:
asdf=pd.DataFrame(a)

In [95]:
asdf

,0,1,2,3,4,5,6,7,8,9,...,54390,54391,54392,54393,54394,54395,54396,54397,54398,54399
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,2,3
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
433,5,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
434,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
435,0,0,0,0,0,0,0,0,0,0,...,3,1,0,0,0,0,0,0,0,0
436,0,0,0,0,0,0,0,0,0,0,...,14,15,15,15,15,19,21,24,24,24


In [96]:
image_data = asdf.iloc[0].values

In [97]:
image_data.shape

(54400,)

### 이미지 증폭

In [98]:
from albumentations import (
    Compose,
    HorizontalFlip,
    VerticalFlip,
    RandomRotate90,
    Rotate,
    GaussNoise,
    RandomScale,
    RandomBrightnessContrast
)


In [99]:
# Albumentations에서 제공하는 여러 가지 증강 기법 중 일부 선택 (예시)

augmenter = Compose([
    HorizontalFlip(p=0.5),
    Rotate(limit=45, p=0.5),  # -45도에서 45도 사이의 임의의 각도로 회전
    GaussNoise(var_limit=(10.0, 50.0), p=0.5),  # 가우시안 노이즈 추가
    RandomBrightnessContrast(brightness_limit=(-0.2, 0.2), contrast_limit=(-0.2, 0.2), p=0.5),  # 밝기와 대비 조절
])

augmented_rows = []

for row_index in range(len(asdf)):
    image_data = asdf.iloc[row_index].values
    image_data_reshaped = image_data.reshape(160, 340)

    # 원본 이미지 저장
    augmented_rows.append(image_data_reshaped.flatten())

    for _ in range(4):  # 원본 포함하여 5배로 증가시키기 위해 4번 반복
        # 증강된 이미지 생성 및 저장 
        augmented_image = augmenter(image=image_data_reshaped)['image']
        augmented_rows.append(augmented_image.flatten())

# DataFrame에 추가하기 위해 numpy array로 변환
augmented_array = np.array(augmented_rows)

# 새 DataFrame 생성 (원본 DataFrame의 컬럼 이름 사용)
df_augmented = pd.DataFrame(data=augmented_array, columns=asdf.columns)

In [100]:
kwangsuDF=df_augmented
kwangsuDF

,0,1,2,3,4,5,6,7,8,9,...,54390,54391,54392,54393,54394,54395,54396,54397,54398,54399
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,4,0,0,0,0,0,3,...,0,2,0,0,0,4,3,6,1,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,4,0,0,0,3,0,0,0,...,3,0,0,10,0,0,0,0,0,7
4,25,22,22,22,23,22,22,22,22,22,...,22,25,22,28,25,23,22,23,22,23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2185,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2186,1,0,7,0,1,0,0,0,0,0,...,0,4,2,2,0,9,0,0,3,5
2187,26,26,26,26,26,26,26,26,26,26,...,26,26,26,26,26,26,26,26,26,26
2188,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 이미지 라벨링

In [101]:
kwangsuDF['man_num']=5

## 영수 데이터 전처리 1

In [102]:
data = np.load('youngsoo.npz')
array2 = data['data1']
array2.shape

(239, 160, 340, 4)

### 4차원으로 배열변환

In [103]:
array2=array2.reshape(239,160, 340, 4)
array2.shape

(239, 160, 340, 4)

### 그레이스케일 후 이미지를 1차원으로 벡터로 변환

In [104]:
a=[]
for i in array2:
    gray = cv2.cvtColor(i, cv2.COLOR_RGBA2GRAY)
    gray=gray.reshape(-1)
    a.append(gray)

In [105]:
asdf=pd.DataFrame(a)
asdf

,0,1,2,3,4,5,6,7,8,9,...,54390,54391,54392,54393,54394,54395,54396,54397,54398,54399
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
234,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
235,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
236,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
237,0,0,0,0,0,0,0,0,0,0,...,93,92,88,86,85,80,77,71,65,57


In [106]:
image_data = asdf.iloc[0].values
image_data.shape

(54400,)

### 이미지 증폭

In [108]:
# Albumentations에서 제공하는 여러 가지 증강 기법 중 일부 선택 (예시)

augmenter = Compose([
    HorizontalFlip(p=0.5),
    Rotate(limit=45, p=0.5),  # -45도에서 45도 사이의 임의의 각도로 회전
    GaussNoise(var_limit=(10.0, 50.0), p=0.5),  # 가우시안 노이즈 추가
    RandomBrightnessContrast(brightness_limit=(-0.2, 0.2), contrast_limit=(-0.2, 0.2), p=0.5),  # 밝기와 대비 조절
])

augmented_rows = []

for row_index in range(len(asdf)):
    image_data = asdf.iloc[row_index].values
    image_data_reshaped = image_data.reshape(160, 340)

    # 원본 이미지 저장
    augmented_rows.append(image_data_reshaped.flatten())

    for _ in range(7):  # 원본 포함하여 5배로 증가시키기 위해 4번 반복
        # 증강된 이미지 생성 및 저장 
        augmented_image = augmenter(image=image_data_reshaped)['image']
        augmented_rows.append(augmented_image.flatten())

# DataFrame에 추가하기 위해 numpy array로 변환
augmented_array = np.array(augmented_rows)

# 새 DataFrame 생성 (원본 DataFrame의 컬럼 이름 사용)
df_augmented = pd.DataFrame(data=augmented_array, columns=asdf.columns)

### 이미지 라벨링

In [109]:
youngsooDF=df_augmented
youngsooDF

,0,1,2,3,4,5,6,7,8,9,...,54390,54391,54392,54393,54394,54395,54396,54397,54398,54399
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,1,0,1,0,0,4,2,4,...,0,0,3,0,0,6,1,4,0,4
2,0,0,0,0,0,3,5,1,0,1,...,5,4,12,0,0,1,7,0,0,0
3,0,0,0,0,5,1,0,0,7,0,...,0,5,0,1,2,1,3,2,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1907,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1908,2,0,5,0,0,0,0,5,0,1,...,0,2,3,4,0,0,0,0,3,5
1909,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1910,0,9,0,0,0,3,0,7,0,0,...,0,3,7,0,0,0,0,5,0,0


In [110]:
youngsooDF['man_num']=1

## 영호 데이터 전처리2

In [111]:
data = np.load('youngho.npz')
array3 = data['data1']
array3.shape

(294, 160, 340, 4)

### 4차원으로 배열변환

In [114]:
array3=array3.reshape(294,160, 340, 4)
array3.shape

(294, 160, 340, 4)

### 그레이스케일 후 이미지를 1차원으로 벡터로 변환

In [117]:
a=[]
for i in array3:
    gray = cv2.cvtColor(i, cv2.COLOR_RGBA2GRAY)
    gray=gray.reshape(-1)
    a.append(gray)

In [118]:
asdf=pd.DataFrame(a)
asdf

,0,1,2,3,4,5,6,7,8,9,...,54390,54391,54392,54393,54394,54395,54396,54397,54398,54399
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
289,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
290,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
291,0,0,0,0,0,0,0,0,0,0,...,69,86,67,19,1,1,0,0,0,1
292,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [80]:
image_data = asdf.iloc[0].values
image_data.shape

(54400,)

### 이미지 증폭

In [119]:
# Albumentations에서 제공하는 여러 가지 증강 기법 중 일부 선택 (예시)

augmenter = Compose([
    HorizontalFlip(p=0.5),
    Rotate(limit=45, p=0.5),  # -45도에서 45도 사이의 임의의 각도로 회전
    GaussNoise(var_limit=(10.0, 50.0), p=0.5),  # 가우시안 노이즈 추가
    RandomBrightnessContrast(brightness_limit=(-0.2, 0.2), contrast_limit=(-0.2, 0.2), p=0.5),  # 밝기와 대비 조절
])

augmented_rows = []

for row_index in range(len(asdf)):
    image_data = asdf.iloc[row_index].values
    image_data_reshaped = image_data.reshape(160, 340)

    # 원본 이미지 저장
    augmented_rows.append(image_data_reshaped.flatten())

    for _ in range(7):  # 원본 포함하여 5배로 증가시키기 위해 4번 반복
        # 증강된 이미지 생성 및 저장 
        augmented_image = augmenter(image=image_data_reshaped)['image']
        augmented_rows.append(augmented_image.flatten())

# DataFrame에 추가하기 위해 numpy array로 변환
augmented_array = np.array(augmented_rows)

# 새 DataFrame 생성 (원본 DataFrame의 컬럼 이름 사용)
df_augmented = pd.DataFrame(data=augmented_array, columns=asdf.columns)

In [120]:
younghoDF=df_augmented
younghoDF

,0,1,2,3,4,5,6,7,8,9,...,54390,54391,54392,54393,54394,54395,54396,54397,54398,54399
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,0,2,0,0,1,0,3,0,0,...,0,1,0,0,5,0,4,0,2,0
2,15,17,15,15,23,17,15,15,15,15,...,15,15,19,19,21,15,17,16,20,15
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,2,0,0,7,0,0,4,...,10,0,7,0,0,3,0,0,9,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2347,43,43,43,43,43,43,43,43,43,43,...,43,43,43,43,43,43,43,43,43,43
2348,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2349,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2350,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 이미지 라벨링

In [121]:
younghoDF['man_num']=2

## 영식 데이터 전처리3

In [122]:
data = np.load('youngsik.npz')
array4 = data['data1']
array4.shape

(308, 160, 340, 4)

### 4차원으로 배열변환

In [124]:
array4=array4.reshape(308,160, 340, 4)
array4.shape

(308, 160, 340, 4)

### 그레이스케일 후 이미지를 1차원으로 벡터로 변환

In [125]:
a=[]
for i in array4:
    gray = cv2.cvtColor(i, cv2.COLOR_RGBA2GRAY)
    gray=gray.reshape(-1)
    a.append(gray)

In [126]:
asdf=pd.DataFrame(a)
asdf

,0,1,2,3,4,5,6,7,8,9,...,54390,54391,54392,54393,54394,54395,54396,54397,54398,54399
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,3
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
303,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
304,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
305,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
306,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [127]:
image_data = asdf.iloc[0].values
image_data.shape

(54400,)

### 이미지 증폭

In [128]:
# Albumentations에서 제공하는 여러 가지 증강 기법 중 일부 선택 (예시)

augmenter = Compose([
    HorizontalFlip(p=0.5),
    Rotate(limit=45, p=0.5),  # -45도에서 45도 사이의 임의의 각도로 회전
    GaussNoise(var_limit=(10.0, 50.0), p=0.5),  # 가우시안 노이즈 추가
    RandomBrightnessContrast(brightness_limit=(-0.2, 0.2), contrast_limit=(-0.2, 0.2), p=0.5),  # 밝기와 대비 조절
])

augmented_rows = []

for row_index in range(len(asdf)):
    image_data = asdf.iloc[row_index].values
    image_data_reshaped = image_data.reshape(160, 340)

    # 원본 이미지 저장
    augmented_rows.append(image_data_reshaped.flatten())

    for _ in range(7):  # 원본 포함하여 5배로 증가시키기 위해 4번 반복
        # 증강된 이미지 생성 및 저장 
        augmented_image = augmenter(image=image_data_reshaped)['image']
        augmented_rows.append(augmented_image.flatten())

# DataFrame에 추가하기 위해 numpy array로 변환
augmented_array = np.array(augmented_rows)

# 새 DataFrame 생성 (원본 DataFrame의 컬럼 이름 사용)
df_augmented = pd.DataFrame(data=augmented_array, columns=asdf.columns)

In [129]:
youngsikDF=df_augmented
youngsikDF

,0,1,2,3,4,5,6,7,8,9,...,54390,54391,54392,54393,54394,54395,54396,54397,54398,54399
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,59,50,50,50,52,50,50,50,50,53,...,50,50,50,50,51,51,50,52,50,50
2,43,45,48,43,43,45,43,44,43,44,...,43,43,43,43,43,51,43,47,44,43
3,0,9,1,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,6,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2459,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2460,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2461,1,2,0,0,0,0,0,0,0,0,...,0,0,6,0,0,2,0,0,1,0
2462,16,16,16,16,16,16,16,16,16,16,...,16,16,16,16,16,16,16,16,16,16


### 이미지 라벨링

In [130]:
youngsikDF['man_num']=3

## 영철 데이터 전처리4

In [131]:
data = np.load('youngchul.npz')
array6 = data['data1']
array6.shape

(296, 160, 340, 4)

### 4차원으로 배열변환

In [132]:
array6=array6.reshape(296,160, 340, 4)
array6.shape

(296, 160, 340, 4)

### 그레이스케일 후 이미지를 1차원으로 벡터로 변환

In [133]:
a=[]
for i in array6:
    gray = cv2.cvtColor(i, cv2.COLOR_RGBA2GRAY)
    gray=gray.reshape(-1)
    a.append(gray)

In [134]:
asdf=pd.DataFrame(a)
asdf

,0,1,2,3,4,5,6,7,8,9,...,54390,54391,54392,54393,54394,54395,54396,54397,54398,54399
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
291,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
292,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
293,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
294,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [135]:
image_data = asdf.iloc[0].values
image_data.shape

(54400,)

### 이미지 증폭

In [136]:
# Albumentations에서 제공하는 여러 가지 증강 기법 중 일부 선택 (예시)

augmenter = Compose([
    HorizontalFlip(p=0.5),
    Rotate(limit=45, p=0.5),  # -45도에서 45도 사이의 임의의 각도로 회전
    GaussNoise(var_limit=(10.0, 50.0), p=0.5),  # 가우시안 노이즈 추가
    RandomBrightnessContrast(brightness_limit=(-0.2, 0.2), contrast_limit=(-0.2, 0.2), p=0.5),  # 밝기와 대비 조절
])

augmented_rows = []

for row_index in range(len(asdf)):
    image_data = asdf.iloc[row_index].values
    image_data_reshaped = image_data.reshape(160, 340)

    # 원본 이미지 저장
    augmented_rows.append(image_data_reshaped.flatten())

    for _ in range(7):  # 원본 포함하여 5배로 증가시키기 위해 4번 반복
        # 증강된 이미지 생성 및 저장 
        augmented_image = augmenter(image=image_data_reshaped)['image']
        augmented_rows.append(augmented_image.flatten())

# DataFrame에 추가하기 위해 numpy array로 변환
augmented_array = np.array(augmented_rows)

# 새 DataFrame 생성 (원본 DataFrame의 컬럼 이름 사용)
df_augmented = pd.DataFrame(data=augmented_array, columns=asdf.columns)

In [137]:
youngchulDF=df_augmented
youngchulDF

,0,1,2,3,4,5,6,7,8,9,...,54390,54391,54392,54393,54394,54395,54396,54397,54398,54399
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,28,28,28,28,34,28,28,28,28,28,...,28,28,28,28,28,28,28,28,28,28
4,51,65,47,49,53,62,47,47,47,47,...,47,47,52,47,48,48,60,47,53,49
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2363,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2364,0,13,8,5,6,0,4,0,0,2,...,0,13,5,2,10,0,0,2,0,0
2365,2,4,2,0,0,9,7,1,0,0,...,0,1,0,0,3,0,0,0,0,4
2366,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 이미지 라벨링

In [138]:
youngchulDF['man_num']=4

## 상철 데이터 전처리6

In [139]:
data = np.load('sangchul.npz')
array7 = data['data1']
array7.shape

(270, 160, 340, 4)

### 4차원으로 배열변환

In [140]:
array7=array7.reshape(270,160, 340, 4)
array7.shape

(270, 160, 340, 4)

### 그레이스케일 후 이미지를 1차원으로 벡터로 변환

In [141]:
a=[]
for i in array7:
    gray = cv2.cvtColor(i, cv2.COLOR_RGBA2GRAY)
    gray=gray.reshape(-1)
    a.append(gray)

In [142]:
asdf=pd.DataFrame(a)
asdf

,0,1,2,3,4,5,6,7,8,9,...,54390,54391,54392,54393,54394,54395,54396,54397,54398,54399
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,46,44,42,40,36,24,24,7,7,7
2,0,0,0,0,0,0,0,0,0,0,...,2,1,1,1,1,1,1,1,1,1
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
265,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
266,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
267,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
268,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [143]:
image_data = asdf.iloc[0].values
image_data.shape

(54400,)

### 이미지 증폭

In [144]:
# Albumentations에서 제공하는 여러 가지 증강 기법 중 일부 선택 (예시)

augmenter = Compose([
    HorizontalFlip(p=0.5),
    Rotate(limit=45, p=0.5),  # -45도에서 45도 사이의 임의의 각도로 회전
    GaussNoise(var_limit=(10.0, 50.0), p=0.5),  # 가우시안 노이즈 추가
    RandomBrightnessContrast(brightness_limit=(-0.2, 0.2), contrast_limit=(-0.2, 0.2), p=0.5),  # 밝기와 대비 조절
])

augmented_rows = []

for row_index in range(len(asdf)):
    image_data = asdf.iloc[row_index].values
    image_data_reshaped = image_data.reshape(160, 340)

    # 원본 이미지 저장
    augmented_rows.append(image_data_reshaped.flatten())

    for _ in range(7):  # 원본 포함하여 5배로 증가시키기 위해 4번 반복
        # 증강된 이미지 생성 및 저장 
        augmented_image = augmenter(image=image_data_reshaped)['image']
        augmented_rows.append(augmented_image.flatten())

# DataFrame에 추가하기 위해 numpy array로 변환
augmented_array = np.array(augmented_rows)

# 새 DataFrame 생성 (원본 DataFrame의 컬럼 이름 사용)
df_augmented = pd.DataFrame(data=augmented_array, columns=asdf.columns)

In [145]:
sangchulDF=df_augmented
sangchulDF

,0,1,2,3,4,5,6,7,8,9,...,54390,54391,54392,54393,54394,54395,54396,54397,54398,54399
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,28,35,27,27,32,27,27,27,28,27,...,28,27,27,28,27,30,27,32,28,27
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2155,3,0,8,0,13,7,0,0,6,8,...,134,120,138,135,134,121,132,122,133,134
2156,0,0,0,0,0,0,0,0,0,0,...,71,69,72,73,72,74,81,83,80,76
2157,0,0,0,0,0,0,0,0,0,0,...,124,155,139,109,96,108,102,109,98,76
2158,0,0,0,0,0,0,0,0,0,0,...,140,139,137,142,148,153,161,169,171,172


### 이미지 라벨링

In [146]:
sangchulDF['man_num']=6

### 데이터 병합 후 넘파이 파일로 저장

In [148]:
manDF=pd.concat([youngsooDF, younghoDF])
manDF=pd.concat([manDF, youngsikDF])
manDF=pd.concat([manDF, youngchulDF])
manDF=pd.concat([manDF, kwangsuDF])
manDF=pd.concat([manDF, sangchulDF])
manDF

,0,1,2,3,4,5,6,7,8,9,...,54391,54392,54393,54394,54395,54396,54397,54398,54399,man_num
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,0,0,1,0,1,0,0,4,2,4,...,0,3,0,0,6,1,4,0,4,1
2,0,0,0,0,0,3,5,1,0,1,...,4,12,0,0,1,7,0,0,0,1
3,0,0,0,0,5,1,0,0,7,0,...,5,0,1,2,1,3,2,0,0,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2155,3,0,8,0,13,7,0,0,6,8,...,120,138,135,134,121,132,122,133,134,6
2156,0,0,0,0,0,0,0,0,0,0,...,69,72,73,72,74,81,83,80,76,6
2157,0,0,0,0,0,0,0,0,0,0,...,155,139,109,96,108,102,109,98,76,6
2158,0,0,0,0,0,0,0,0,0,0,...,139,137,142,148,153,161,169,171,172,6


In [149]:
# manDF.to_csv("manDF.csv")

In [150]:
np.save('manDF.npy', manDF)